### Read MySQL Db using Pyspark:

In [22]:

from pyspark.sql import SparkSession
spark = SparkSession\
    .builder\
    .appName("read-mysqldb-pyspark")\
    .config("spark.jars.packages", "com.mysql:mysql-connector-j:8.3.0")\
    .getOrCreate()
    # .config("spark.driver.extraClassPath", "/home/tuhin/mysql.jar")\

dataframe_mysql = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:mysql://localhost/gdb041")\
    .option("driver", "com.mysql.jdbc.Driver")\
    .option("dbtable", "dim_customer").option("user", "root")\
    .option("password", "12345678").load()

dataframe_mysql.show(2)

+------------------+------+--------------+--------+-------------+
|          customer|market|      platform| channel|customer_code|
+------------------+------+--------------+--------+-------------+
|  Electricalsocity| India|Brick & Mortar|Retailer|     90002012|
|Electricalslytical| India|Brick & Mortar|Retailer|     90002013|
+------------------+------+--------------+--------+-------------+
only showing top 2 rows



### List all schemas from MySql Db:
- **SHOW SCHEMAS is not allowed in a subquery**
    - dbtable/table can be either a table/view name or a SELECT subquery wrapped in parentheses and aliased.
    - SHOW statements are not valid inside subqueries. Use the metadata table instead:
        `SELECT schema_name FROM information_schema.schemata.`

In [23]:
jdbc_url = "jdbc:mysql://localhost:3306/information_schema?useSSL=false&allowPublicKeyRetrieval=true"

properties = {
    "user": "root",
    "password": "12345678",
    "driver": "com.mysql.cj.jdbc.Driver"
}

# Use a SELECT subquery wrapped in parentheses and give it an alias
query = """
(
  SELECT schema_name AS database_name
  FROM information_schema.schemata
  ORDER BY schema_name
) AS t
"""

In [24]:
df = spark.read.jdbc(
    url=jdbc_url,
    table=query,     
    properties=properties
)

df.show(truncate=False)
df.printSchema()

+------------------+
|database_name     |
+------------------+
|adventureworks    |
|b12_dataengineer  |
|enquiry_d         |
|formdb            |
|gdb041            |
|gdb056            |
|home              |
|information_schema|
|myproject_dev     |
|myschema          |
|mysql             |
|mysql_practice    |
|newschema         |
|newschema2        |
|org               |
|performance_schema|
|practice1         |
|sakila            |
|sales             |
|sql_rampup        |
+------------------+
only showing top 20 rows

root
 |-- database_name: string (nullable = true)



In [25]:
jdbc_url2 = "jdbc:mysql://localhost:3306/information_schema?useSSL=true&allowPublicKeyRetrieval=true"

df = spark.read.jdbc(
    url=jdbc_url2,
    table=query,     
    properties=properties
)

df.show(truncate=False)
df.printSchema()

+------------------+
|database_name     |
+------------------+
|adventureworks    |
|b12_dataengineer  |
|enquiry_d         |
|formdb            |
|gdb041            |
|gdb056            |
|home              |
|information_schema|
|myproject_dev     |
|myschema          |
|mysql             |
|mysql_practice    |
|newschema         |
|newschema2        |
|org               |
|performance_schema|
|practice1         |
|sakila            |
|sales             |
|sql_rampup        |
+------------------+
only showing top 20 rows

root
 |-- database_name: string (nullable = true)



### Read MySQL Db using Python:

In [34]:

import mysql.connector

DB_HOST = "localhost"
DB_PORT = 3306
DB_USER = "root"       
DB_PASSWORD = "12345678"  
# DB_NAME = "gdb041"

def mysql_conn(database_name):
    conn = mysql.connector.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=database_name,
    )
    return conn

with mysql_conn(database_name="gdb041") as conn:
    with conn.cursor() as cur:
        cur.execute("""
            SHOW TABLES;
        """)
        for row in cur.fetchall():
            print(row[0])

dim_customer
dim_market
dim_product
fact_forecast_monthly
fact_sales_monthly


In [49]:
def execute_query(database_name, table_name):
    with mysql.connector.connect(
        host="127.0.0.1",
        port="3306",
        user="root",
        password="12345678",
        database=database_name,
    ) as conn:
        with conn.cursor() as cur:
            cur.execute(f"SELECT COUNT(*) FROM {database_name}.{table_name}")
            count = [row[0] for row in cur.fetchall()]
            return count

with mysql_conn(database_name="myschema") as conn:
    with conn.cursor() as cur:
        cur.execute("""
            SHOW TABLES;
        """)
        for row in cur.fetchall():
            print(row[0])
            print(execute_query(database_name="myschema", table_name=f"{row[0]}"))

2015_summary
[255]
2015_summary_csv
[256]
2015_summary_json
[256]
2015_summary_parquet
[255]


In [48]:
df = spark.read.option("multiLine", "false").option("inferSchema", "true").json(r"D:\GitLocal\big_data\Dataset\2015-summary.json")
df.show(truncate=False)
df.printSchema()

+------------------------+-------------------+-----+
|DEST_COUNTRY_NAME       |ORIGIN_COUNTRY_NAME|count|
+------------------------+-------------------+-----+
|United States           |Romania            |15   |
|United States           |Croatia            |1    |
|United States           |Ireland            |344  |
|Egypt                   |United States      |15   |
|United States           |India              |62   |
|United States           |Singapore          |1    |
|United States           |Grenada            |62   |
|Costa Rica              |United States      |588  |
|Senegal                 |United States      |40   |
|Moldova                 |United States      |1    |
|United States           |Sint Maarten       |325  |
|United States           |Marshall Islands   |39   |
|Guyana                  |United States      |64   |
|Malta                   |United States      |1    |
|Anguilla                |United States      |41   |
|Bolivia                 |United States      |